In [1]:
from fastai.vision.all import *
from fastai.distributed import *
from fastai.metrics import error_rate
from fastai.callback.tracker import SaveModelCallback
import argparse

from torchvision import datasets, transforms, models
import torch.optim as optim
from torch.optim import lr_scheduler
import time
import os
import copy
import torchvision.transforms as T
import torch
from torch.nn.parallel import DistributedDataParallel
from torchvision.transforms.functional import *

from PIL import Image
import requests

import matplotlib.pyplot as plt
import numpy as np
import torch.nn.functional as F
from torch import nn

import argparse
from models.utils.joiner3 import ImageNetJoiner
from models.utils.new_losses import *
from models.utils.metrics import *
from models.utils.dataLoader import *
from models.utils.datasets import *

In [2]:
torch.cuda.set_device("cuda:3")
device = torch.device("cuda:3" if torch.cuda.is_available() else "cpu")

In [3]:
H = 256
W= 256
bs = 5

In [4]:
def get_gm(r):
    label = parent_label(r)
    a = attrgetter("name")
    rgex = RegexLabeller(pat = r'image(.*?).jpeg') 
    gm = torch.load(save_path+"/gramm/"+str(label)+"/gm"+rgex(a(r))+".pt")
    return gm, TensorCategory(int(label))

In [5]:
def paths(name = 'Layer3', dataset = 'IMAGENETTE'): 
    model_dir = Path.home()/'Luiz/saved_models/paper/finetuned'
    wname = name+'_'+dataset+'.pth'
    weights_path = model_dir/wname
    if dataset == 'IMAGENETTE':
        data_path = untar_data(URLs.IMAGENETTE)
    elif dataset == 'IMAGEWOOF':
        data_path = untar_data(URLs.IMAGEWOOF)
    elif dataset == 'CIFAR':
        data_path = untar_data(URLs.CIFAR)
    elif dataset == 'CIFAR100':
        data_path = untar_data(URLs.CIFAR_100)  
    elif dataset == 'CALTECH_101':    
        data_path = untar_data(URLs.CALTECH_101) 
    elif dataset == 'FLOWERS':
        data_path = untar_data(URLs.FLOWERS)
    else:
        data_path = None
        
    return weights_path, data_path

In [6]:
def get_gm(r):
    label = parent_label(r)
    a = attrgetter("name")
    rgex = RegexLabeller(pat = r'image(.*?).jpeg') 
    gm = torch.load(save_path+"/gramm/"+str(label)+"/gm"+rgex(a(r))+".pt")
    return gm, TensorCategory(int(label))

def load_model():
    model_path = Path.home()/'Luiz/saved_models/paper/ARViT-Base.pkl'
    net = load_learner(model_path, cpu=False)
    model = net.model
        
    return model

def model_head(model, n_classes):
    model.head = nn.Linear(512*16*16, n_classes)
    #model.noise_mode = True
    #model.generator_mode = False

    trainable = ['head.weight','head.bias']
    for name, p in model.named_parameters():
        if name not in trainable:
            p.requires_grad = False
        else:
            p.requires_grad = True
            
def data_loader(path):
    transform = ([*aug_transforms(),Normalize.from_stats([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])

    data = DataBlock(blocks=(ImageBlock, CategoryBlock), 
                     get_items=get_image_files, 
                     splitter=RandomSplitter(),
                     get_y=parent_label,
                     item_tfms=Resize(H,W),
                     batch_tfms=transform)

    dloader = data.dataloaders(path,bs=bs)
    return dloader

def flowers_loader(path):
    Path.BASE_PATH = path
    path.ls()
    df = pd.read_csv('data/flowers.csv')

    transform = ([*aug_transforms(),Normalize.from_stats([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])

    def get_x(r): return path/r['name']
    def get_y(r): return r['class']

    dblock = DataBlock(blocks    = (ImageBlock, CategoryBlock),
                       n_inp=1,
                       splitter=RandomSplitter(seed=42),
                       get_x= get_x,
                       get_y= get_y, 
                       item_tfms = Resize(256),
                       #batch_tfms= Normalize.from_stats(*imagenet_stats)
                       batch_tfms= transform
                      )

    dloader = dblock.dataloaders(df, bs=bs)
    return dloader

In [7]:
def model_test(n_class, name='Layer3', dataset='IMAGENETTE', epochs=1, lr=1e-9):
    weights_path, path_data = paths(name, dataset)
    #print(path_model, weights_path, path_data)
    model = load_model()
    model_head(model, n_class)
    print("path model:", weights_path)
    weight_dict = load_learner(weights_path, cpu=False)
    model.load_state_dict(weight_dict)
    print("weights loaded")
        
    if dataset == 'FLOWERS':
        dloader = flowers_loader(path_data)
    else:
        dloader = data_loader(path_data)
    #Defining the Loss Function
    critic_loss = SingleLabelCriticLoss()
    
    #Wraping the Learner
    learner = Learner(dloader, model, loss_func=critic_loss, metrics=[Accuracy])
    learner.fit_one_cycle(epochs, lr)
    #learner.fine_tune(60, base_lr=5e-4, freeze_epochs=30)

In [9]:
weights_path, path_data = paths('Layer3', 'IMAGENETTE')
#print(path_model, weights_path, path_data)
model = load_model()
model_head(model, n_class)
print("path model:", weights_path)
weight_dict = load_learner(weights_path, cpu=False)
model.load_state_dict(weight_dict)
print("weights loaded")

NameError: name 'n_class' is not defined

In [8]:
model_test(n_class=102, name='ARViT-L1-G32', dataset='FLOWERS')

path model: /home/atsumilab/Luiz/saved_models/paper/finetuned/ARViT-L1-G32_FLOWERS.pth
weights loaded


epoch,train_loss,valid_loss,Accuracy,time
0,0.131137,0.606523,0.837508,00:36


In [9]:
model_test(n_class=102, name='ARViT-L1-G32_Best', dataset='FLOWERS')

path model: /home/atsumilab/Luiz/saved_models/paper/finetuned/ARViT-L1-G32_Best_FLOWERS.pth
weights loaded


epoch,train_loss,valid_loss,Accuracy,time
0,0.133211,0.611318,0.831399,00:36


In [10]:
model_test(n_class=102, name='ARViT-L6-G32', dataset='FLOWERS')

path model: /home/atsumilab/Luiz/saved_models/paper/finetuned/ARViT-L6-G32_FLOWERS.pth
weights loaded


epoch,train_loss,valid_loss,Accuracy,time
0,0.142949,0.581903,0.849114,00:37


In [11]:
model_test(n_class=102, name='ARViT-L6-G32_Best', dataset='FLOWERS')

path model: /home/atsumilab/Luiz/saved_models/paper/finetuned/ARViT-L6-G32_Best_FLOWERS.pth
weights loaded


epoch,train_loss,valid_loss,Accuracy,time
0,0.163460,0.599245,0.846671,00:36


In [14]:
model_test(n_class=100, name='ARViT-L1-G32', dataset='CIFAR100')

path model: /home/atsumilab/Luiz/saved_models/paper/finetuned/ARViT-L1-G32_CIFAR100.pth
weights loaded


epoch,train_loss,valid_loss,Accuracy,time
0,0.939998,0.576333,0.848500,03:58


In [15]:
model_test(n_class=100, name='ARViT-L1-G32_Best', dataset='CIFAR100')

path model: /home/atsumilab/Luiz/saved_models/paper/finetuned/ARViT-L1-G32_Best_CIFAR100.pth
weights loaded


epoch,train_loss,valid_loss,Accuracy,time
0,0.942338,0.605219,0.840917,03:57


In [16]:
model_test(n_class=100, name='ARViT-L6-G32', dataset='CIFAR100')

path model: /home/atsumilab/Luiz/saved_models/paper/finetuned/ARViT-L6-G32_CIFAR100.pth
weights loaded


epoch,train_loss,valid_loss,Accuracy,time
0,0.951208,0.593923,0.847417,03:58


In [17]:
model_test(n_class=100, name='ARViT-L6-G32_Best', dataset='CIFAR100')

path model: /home/atsumilab/Luiz/saved_models/paper/finetuned/ARViT-L6-G32_Best_CIFAR100.pth
weights loaded


epoch,train_loss,valid_loss,Accuracy,time
0,0.913626,0.584167,0.848000,03:58


In [18]:
model_test(n_class=10, name='ARViT-L1-G32', dataset='IMAGENETTE')

path model: /home/atsumilab/Luiz/saved_models/paper/finetuned/ARViT-L1-G32_IMAGENETTE.pth
weights loaded


epoch,train_loss,valid_loss,Accuracy,time
0,0.287862,0.170025,0.948096,00:52


In [19]:
model_test(n_class=10, name='ARViT-L1-G32_Best', dataset='IMAGENETTE')

path model: /home/atsumilab/Luiz/saved_models/paper/finetuned/ARViT-L1-G32_Best_IMAGENETTE.pth
weights loaded


epoch,train_loss,valid_loss,Accuracy,time
0,0.238245,0.161538,0.951830,00:52


In [20]:
model_test(n_class=10, name='ARViT-L6-G32', dataset='IMAGENETTE')

path model: /home/atsumilab/Luiz/saved_models/paper/finetuned/ARViT-L6-G32_IMAGENETTE.pth
weights loaded


epoch,train_loss,valid_loss,Accuracy,time
0,0.300479,0.175379,0.945108,00:52


In [21]:
model_test(n_class=10, name='ARViT-L6-G32_Best', dataset='IMAGENETTE')

path model: /home/atsumilab/Luiz/saved_models/paper/finetuned/ARViT-L6-G32_Best_IMAGENETTE.pth
weights loaded


epoch,train_loss,valid_loss,Accuracy,time
0,0.269931,0.184755,0.944735,00:52


In [22]:
model_test(n_class=10, name='ARViT-L1-G32', dataset='CIFAR')

path model: /home/atsumilab/Luiz/saved_models/paper/finetuned/ARViT-L1-G32_CIFAR.pth
weights loaded


epoch,train_loss,valid_loss,Accuracy,time
0,0.479238,0.397447,0.861167,03:38


In [23]:
model_test(n_class=10, name='ARViT-L1-G32_Best', dataset='CIFAR')

path model: /home/atsumilab/Luiz/saved_models/paper/finetuned/ARViT-L1-G32_Best_CIFAR.pth
weights loaded


epoch,train_loss,valid_loss,Accuracy,time
0,0.543407,0.385890,0.869750,03:43


In [24]:
model_test(n_class=10, name='ARViT-L6-G32', dataset='CIFAR')

path model: /home/atsumilab/Luiz/saved_models/paper/finetuned/ARViT-L6-G32_CIFAR.pth
weights loaded


epoch,train_loss,valid_loss,Accuracy,time
0,0.526396,0.372448,0.871000,03:39


In [25]:
model_test(n_class=10, name='ARViT-L6-G32_Best', dataset='CIFAR')

path model: /home/atsumilab/Luiz/saved_models/paper/finetuned/ARViT-L6-G32_Best_CIFAR.pth
weights loaded


epoch,train_loss,valid_loss,Accuracy,time
0,0.468442,0.369256,0.874000,03:40


In [26]:
model_test(n_class=10, name='ARViT-L1-G32', dataset='IMAGEWOOF')

path model: /home/atsumilab/Luiz/saved_models/paper/finetuned/ARViT-L1-G32_IMAGEWOOF.pth
weights loaded


epoch,train_loss,valid_loss,Accuracy,time
0,0.825701,0.611274,0.802703,00:50


In [27]:
model_test(n_class=10, name='ARViT-L1-G32_Best', dataset='IMAGEWOOF')

path model: /home/atsumilab/Luiz/saved_models/paper/finetuned/ARViT-L1-G32_Best_IMAGEWOOF.pth
weights loaded


epoch,train_loss,valid_loss,Accuracy,time
0,0.808609,0.618858,0.801158,00:51


In [28]:
model_test(n_class=10, name='ARViT-L6-G32', dataset='IMAGEWOOF')

path model: /home/atsumilab/Luiz/saved_models/paper/finetuned/ARViT-L6-G32_IMAGEWOOF.pth
weights loaded


epoch,train_loss,valid_loss,Accuracy,time
0,0.733986,0.617505,0.799614,00:50


In [29]:
model_test(n_class=10, name='ARViT-L6-G32_Best', dataset='IMAGEWOOF')

path model: /home/atsumilab/Luiz/saved_models/paper/finetuned/ARViT-L6-G32_Best_IMAGEWOOF.pth
weights loaded


epoch,train_loss,valid_loss,Accuracy,time
0,0.785065,0.599061,0.808108,00:51


In [13]:
def paths2(layer = 1, dataset = 'IMAGENETTE', base_model=False, best_model=False): 
    model_dir = Path.home()/'Luiz/saved_models'
    if base_model == True:
        model_path = model_dir/'GramImageNet_Rotation_16x16grid_epochs-90_BaseModel.pkl'
    else:
        file_name = 'GramImageNet_Rotation_16x16grid_epochs-90-beta-5e-6_PenaltyFactor3_Layer'+str(layer)+'.pkl'
        model_path = model_dir/file_name
        
    if best_model == True:
        weight_dir = Path.home()/'Luiz/gan_attention/models/pretrained/'
        if base_model == True:
            file_name = 'Best_Model_BaseModel_ImageNet_Rotation_16x16grid.pth'
        else:
            file_name = 'Best_Model_Layer'+str(layer)+'_ImageNet_Rotation_16x16grid.pth'
        weights_path = weight_dir/file_name
    else:
        weights_path = None
    
    if dataset == 'IMAGENETTE':
        data_path = untar_data(URLs.IMAGENETTE)
    elif dataset == 'IMAGEWOOF':
        data_path = untar_data(URLs.IMAGEWOOF)
    elif dataset == 'CIFAR':
        data_path = untar_data(URLs.CIFAR)
    elif dataset == 'CIFAR_100':
        data_path = untar_data(URLs.CIFAR_100)    
    elif dataset == 'CALTECH_101':    
        data_path = untar_data(URLs.CALTECH_101) 
    else:
        data_path = None
        
    return model_path, weights_path, data_path

In [23]:
def data_loader(path):
    transform = ([*aug_transforms()])#,Normalize.from_stats([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])

    data = DataBlock(blocks=(ImageBlock, CategoryBlock), 
                     get_items=get_image_files, 
                     splitter=RandomSplitter(),
                     get_y=parent_label,
                     item_tfms=Resize(H,W),
                     batch_tfms=transform)

    dloader = data.dataloaders(path,bs=bs)
    return dloader

In [16]:
def fine_tune_model(n_class, layer = 1, dataset = 'IMAGENETTE', base_model=False, best_model=False, epochs=[1,1], lr=5e-4):
    path_model, weights_path, path_data = paths2(layer, dataset, base_model, best_model)
    print(path_model, weights_path, path_data)
    print("Training on", dataset)
    model = load_model(path_model, weights_path)
    model_head(model, n_class)
        
    dloader = data_loader(path_data)
    #Defining the Loss Function
    critic_loss = SingleLabelCriticLoss()  

    #Wraping the Learner
    learner = Learner(dloader, model, loss_func=critic_loss, metrics=[_Accuracy])
    #learner.fit_one_cycle(50, 0.002)
    learner.fine_tune(epochs[0], base_lr=lr, freeze_epochs=epochs[1])

In [68]:
data_path = untar_data(URLs.FOOD)

In [69]:
(data_path).ls()

(#8) [Path('/home/atsumilab/.fastai/data/food-101/train.json'),Path('/home/atsumilab/.fastai/data/food-101/test.json'),Path('/home/atsumilab/.fastai/data/food-101/images'),Path('/home/atsumilab/.fastai/data/food-101/labels.txt'),Path('/home/atsumilab/.fastai/data/food-101/test.txt'),Path('/home/atsumilab/.fastai/data/food-101/h5'),Path('/home/atsumilab/.fastai/data/food-101/train.txt'),Path('/home/atsumilab/.fastai/data/food-101/classes.txt')]

In [66]:
transform = ([*aug_transforms()])#,Normalize.from_stats([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])

data = DataBlock(blocks=(ImageBlock, CategoryBlock), 
                 get_items=get_image_files, 
                 splitter=RandomSplitter(),
                 get_y=using_attr(RegexLabeller(r'(?<=image_)(.*)(?=.jpg)'),'name'),
                 item_tfms=Resize(256,256),
                 batch_tfms=transform)

dloader = data.dataloaders(data_path/"jpg", bs=5)

Could not do one pass in your dataloader, there is something wrong in it


In [67]:
files = get_image_files(data_path/"jpg")
len(files)

8189

'009'